In [1]:
from IPython.display import HTML, display
display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))
import sales_models as sm

In [2]:
%%capture
X = sm.PriorsPlayML(11)

# Introduction

In this Play-ML notebook we are going to explore the concept of **uncertainty quantification** in the context of choosing a price to set for a product. One reason that decisions are difficult to take is because of uncertainty: can we be sure that *this* is the right thing to do and not *that*? What's the risk of getting things wrong? Let's illustrate uncertainty quantification in the context of pricing.

Let's suppose we own a website that sells exactly one kind of product&mdash;a deluxe mud kitchen, say&mdash;and that any customer will only buy at most one (you really only need one mud kitchen, *deluxe* or otherwise). In this notebook we will think about a simple model for how sales of our mud kitchen depend on the price we set.

## A toy model

The model is described by three parameters. There is an **arrival rate**, at which potential customers stumble across our site. For simplicity we don't assume that our price is marketed or scraped from our website: the arrival rate is fixed at a number of customers per day and doesn't vary with price.

Where price does enter the picture is in the conversion rate: the probability that any one customer on our website will buy a mud kitchen. This is assumed to be very much price dependent: when the price is low, this probability is almost 100%, but as the price increases the probability of a purchase will decrease to zero. We model this behaviour with a logistic curve, which gives two degrees of freedom to set. The first of these is the **reference price**, which can be defined as the price at which the probability of a customer making a purchase is 50%.

The second parameter characterising the price response is the **reference elasticity**. This parameter, denoted by $E$, can be interpreted as price sensitivity: if we put the price at a 1% premium on the reference price, then demand will increase by $E$%. Of course, we would expect demand to *decrease*, which is why elasticity is usually negative. 

These three numbers are enough to fully characterise our model, the outputs of which are shown below. You can play around with the values of these parameters and observe what happens to the model outputs.

In [3]:
X.visualise_nominal_model()

The first output, in the top left, is the **Purchase probability** curve in response to price. This follows the rule of decreasing from almost 100% to almost zero as price increases beyond the reference price, but the precise shape of this curve can be influenced by both the reference price and reference elasticity.

The top right curve is the **elasticity of demand** at different prices. When above -1, people are less sensitive to prices, and price increases will not decrease demand very much. When below -1, people are more sensitive to prices, and price decreases will increase demand a lot. The balance is found at elasticity of -1.

The bottom left curve represents the daily **sales** at each price, where you can observe a ribbon around the green curve. This is because our model incorporates randomness: we have arrivals at a given rate, and purchases at a given probability. These convert to a random number of sales. The curve shows the long-run average of this distribution, while the ribbon shows the likely values of realised sales on any one day. 

The bottom right curve translates the sales into **revenue**. Again, there is a ribbon around the red curve: the revenue in any one day is likely to fall somewhere within the ribbon, but in the long run the daily revenue at any price will take the corresponding value of the red curve. The key observation to make here is that we can select a price for which the expected revenue has a peak. This price is exactly the price at which elasticity is equal to -1, as identified in the top right. This means that if we know the parameters of the model, we can translate that into an optimal decision: we can set the price to maximise the expected revenue. The revenue-maximising price is recorded in the top right of the figure above. You can observe how the optimal price varies with the model parameters. 

An important observation can be made here: the arrival rate has no impact on the correct decision. However, the *value* of ensuring the correct decision does change: the revenue curve scales with the arrival rate, and so the benefits of getting the decision correct will increase.

# Learning model parameters from data

In the exploration above, the model parameters could all be set by playing around with the sliders. However, we want our model to be reflective of reality. While we can set the price, the arrival rate, reference price and reference elasticity are all quantities that are characteristic to our market, and aren't known *a priori*. We've observed how the optimal decision is dependent on the parameter values, and so we need to learn them from data.

Here, we've generated some synthetic sales data from our model, with parameters set. We can play around with the prices to produce new data sets, but all of the data is produced based on the same, fixed, parameter values.

In [4]:
X.visualise_data()

The figures below show a typical approach to learning the parameters of a model. The three parameters are chosen to maximise the *likelihood* of the data. That is, we find values of the three parameters such that the probability of the observed data under the resulting model is highest. This is known as the maximum likelihood estimate, or MLE. We learn sequentially, refining our learning over each of the seven days, based only on the data observed up to that point. So, the learned parameter values on Monday are based only on Monday's data; on Tuesday, they are based on both Monday and Tuesday's data; and so on, until the learned parameter values on Sunday are based on the entire week's data. 

In [5]:
X.visualise_MLEs()

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1500x480 with 3 Axes>', '…

On each day, we see how the three parameter values are estimated based on the supplied data. The left-hand plot compares the three found parameter values against the true parameter values that actually generated the synthetic data. Depending on the data supplied, these can be relatively close or worryingly far away from the true values. The middle plot shows that the data is slightly more likely under the MLE parameter values than under the true values. Finally, the right-hand plot shows the impact of the estimated parameter values on the corresponding decisions: depending on the synthetic data supplied, these are sometimes near optimal, but sometimes concerningly far away.

Note that these figures are only available to us because we know the true parameter values that generated the data. It is worth reiterating that these are not known!

The key observation here is that we are not quantifying *uncertainty*. The MLE parameter values are produced by an optimisation, resulting in a pricing decision. Both the parameters and the resulting price are point estimates only. The middle plot makes it clear that by *optimising* likelihood, we are simply discarding different parameter values that also give a very good fit to data&mdash;not least, the *true* parameter values! We would like to be more open minded about all of those parameter values that also fit the data very well.

What's more, the MLE parameter values are not stable; as more data is supplied, they are updated, sometimes quite significantly. One might assume that more data will make our estimate more reliable, but there is not much indication of *how* much more reliable our decision will be on Sunday than it was on Tuesday. 

# Uncertainty

We will now explore what we mean by uncertainty. Recall that in the first figure above, we set the three parameter values. These parameter values generated an optimal price, in terms of maximising the long-term expected daily revenue.

Now suppose we only roughly know the parameter values, in the sense that we know the arrival rate is near 1000, that the reference price is near £50, and that the reference elasticity is near -1.5. How near? We will quantify this by a scaling parameter that represents the degree of uncertainty in these values. We put a *distribution* onto parameter space, that represents our uncertainty. Because of this distribution, we can make statements that *quantify* our uncertainty. For example, suppose we set the top slider to 50. Then we are specifying that we are 70% sure that the arrival rate is between 950 and 1050.

In [6]:
X.visualise_prior()

If we know the parameters, then we know the optimal decision. If the parameters are uncertain, then the optimal decision is uncertain. Having quantified the uncertainty in the parameters, this means we can quantify the uncertainty in the resulting decision. The purple distribution in the plot above shows how the distribution in the parameters is mapped to a distribution in the optimal decision, so that it is possible to quantify the uncertainty in our decision-making.

Below, we show how this uncertainty propagates into all aspects of the model: as the parameters vary within their distribution, the price-dependent values of purchase probability, elasticity, expected sales and expected revenue all reflect this uncertainty. The ribbons on each curve reflect our uncertainty: darker ribbons represent 50% of the uncertainty and lighter ribbons 90% of the uncertainty. The solid curves here represent the medians of each distribution. Depending on how uncertain we are in our parameter values, the model outputs can be very uncertain indeed.

In [7]:
X.visualise_prior_predictive()

# Taming uncertainty with data

Before introducing uncertainty, we learned some maximum likelihood estimates from data. These produced point values for each of the parameters, based on the results of optimisation. Now, we are going to work with distributions rather than single points, because we are answering a different question: how is uncertainty changing in the context of data?

## Thomas Bayes

The approach to this question is named after an 18th century English nonconformist minister, The Reverend Thomas Bayes, who was a mathematically inclined member of the Royal Society. A paper of his, posthumously read to the Royal Society, is the source of Bayes' theorem. This is the mathematical basis of the *Bayesian* approach to combining uncertainty, represented as a *prior distribution*, with data, to produce a *posterior distribution*. By quirk of location, Bayes and Datasparq have an interesting affinity. As a young man assisting his father, also a nonconformist minister, Bayes worked at nearby Leather Lane and Salters' Hall, and he is now buried in Bunhill Fields&mdash;all within a few minutes' walk of the Datasparq offices in Clerkenwell.

## Sequential Bayesian learning

Recall the data we used earlier to acquire the MLEs.

In [8]:
X.visualise_data()

As with the MLE approach, we proceed day by day and sequentially update our uncertainty in the context of each day's data. Now, though, the output from each day is a distribution rather than a point estimate. In the figures below, the vertical dotted lines represent the true parameter values and optimal price: that is, the parameters corresponding to the generation of the synthetic data used in our learning procedure. Likelihoods are still used, but now near-optimal parameter values are not thrown away: just reweighted accordingly. As each day's data is incorporated into our knowledge, the distributions representing our uncertainty change.

In [9]:
X.visualise_posterior()

Having incorporated our uncertainty with the synthetic data, the purple distribution above represents the remaining uncertainty in the optimal decision. In addition to a pricing strategy (i.e. pick the price where this distribution peaks, or pick a price at random from this distribution) we have also quantified the level of confidence we can feel in this price being optimal.

Below, we once again show how our posterior uncertainty, conditioned on data, propagates into the model outputs. Again, the black dotted lines refer to the ground truth. Having learnt posterior distributions from combining our prior uncertainty with data, we can observe how our uncertain estimates of the model outputs accord with this reality. Of course, this reality is not knowable in general!

In [10]:
X.visualise_posterior_predictive()

## A muddy toy model

A mud kitchen is a muddy toy. So too is the model explored in this notebook, which has been deliberately chosen to be difficult to identify from the supplied data. Suppose we see 1000 sales. Is that 5000 arrivals with a purchase probability of 20% or 1250 arrivals at 80%? By only supplying sales data, we cannot unpick those two numbers. This concept is known as *parameter unidentifiability*. A similar phenomenon happens when our synthetic data is generated using only one price: how is price sensitivity going to be estimated without observing sales at different prices?

When only considering the MLE approach, this unidentifiability is not obvious from the outputs. An estimate is produced regardless, with no indication of how good it might be. However, when looking at our Bayesian posterior distributions, we see that the posterior distribution of the arrival rate is often more spread out than the prior, in the sense of being less peaked. This is an important indicator that our data is insufficient for identifying the model. Nevertheless, the posterior predictive distribution of the revenue as a function of price tends to be a good match to reality, and a good decision can often be made even when the true parameter is in the tails of the posteriors.

## Priors and subjectivity in Bayesian analysis

Another important distinction between a Bayesian approach and the MLE approach is the use of *priors*: the quantification of uncertainty upfront, before incorporating any data. This is a subjective judgement call. Used well, this is a strength of the Bayesian approach; used badly, there is the potential to cause significant damage. A key problem is overconfidence. Nothing stops us overasserting ourselves at the start of the process, and the Bayesian approach does not immediately identify overconfidence. If we push all of the sliders representing uncertainty to the left, we are saying that we are very confident about the parameter values&mdash;even though we are wrong. This confidence tends to override the contradictory information provided by the small amount of data used in this toy example.

The design of priors is extremely important to powerful uncertainty quantification methods. Well-designed, open-minded but accurate priors can overcome the parameter unidentifiability problem by incorporating expertise and tacit knowledge where no data is available; designed badly, they can lead to severely suboptimal, overconfident recommendations. Prior design is one aspect of Bayesian analysis where close collaboration between data scientists and domain experts is absolutely vital.

## Extending the models

There is some passivity in going from our first figure, where the parameters were set, to the subsequent discussion of learning 'true' parameters. The real world is somewhere in-between. We might well have some real parameters, but other decisions we take can affect these. External marketing, website copy, an improved product (a *super*-deluxe mud kitchen!) can all affect these parameters, but measuring these effects can only be done in the context of quantified uncertainty. Different customer types (parents vs grandparents, for example) may well have quite different reference prices and price sensitivities, but to what extent? What data do we need for these to be identifiable? Are they different enough that we can detect them in the context of uncertainty? And of course time plays an important role. How do reference prices and price sensitivities change in the bigger economic context?
